In [2]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.4 MB/s eta 0:00:0000:0100:010m


In [7]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta

In [91]:
conn = mysql.connector.connect(
    host='mysql_container',
    user='root',    
    password='root',
    database='mysql_bd2'
)
cursor = conn.cursor()

### Clientes

In [79]:
fake = Faker('pt_BR')

clientes = []

for _ in range(1000):
    nome = fake.name()
    cpf = fake.numerify(text='###########')
    data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=90)
    endereco = fake.address()
    telefone = fake.phone_number()
    telefone = ''.join(filter(str.isdigit, telefone))
    email = fake.email()
    clientes.append((nome, cpf, data_nascimento.strftime('%Y-%m-%d'), endereco, telefone, email,))
    # print(data_nascimento)

sql = '''
INSERT INTO clientes (Nome, CPF, DataNascimento, Endereco, Telefone, Email)
VALUES (%s, %s, %s, %s, %s, %s);
'''

('Rodrigo Castro', '20581598899', '1936-04-02', 'Praça Pinto, 32\nSanta Branca\n20167060 Azevedo Verde / RO', '555144729748', 'sophielima@example.com')


In [80]:
for i, clt in enumerate(clientes):
    if not all(isinstance(item, str) for item in clt):
        print(f"Erro no cliente {i}: {clt}")

In [81]:
cursor.executemany(sql, clientes)
conn.commit()

print("[] - Inserção de clientes concluída.")

[] - Inserção de clientes concluída.


In [82]:
cursor.execute('SELECT * FROM clientes;')
clientes = cursor.fetchall()
for cliente in clientes:
    print(cliente)

(1, 'Rodrigo Castro', '20581598899', datetime.date(1936, 4, 2), 'Praça Pinto, 32\nSanta Branca\n20167060 Azevedo Verde / RO', '555144729748', 'sophielima@example.com', datetime.datetime(2024, 6, 27, 21, 26, 10), None)
(2, 'Ana Lívia da Mota', '82463930624', datetime.date(1989, 12, 23), 'Sítio de Borges, 19\nAlto Caiçaras\n84191950 Fonseca de Minas / RR', '08006994645', 'yferreira@example.net', datetime.datetime(2024, 6, 27, 21, 26, 10), None)
(3, 'Otávio Pereira', '27766076387', datetime.date(1934, 4, 3), 'Condomínio Ana Vieira, 218\nMorro Dos Macacos\n75634768 Farias / RN', '5503183060090', 'agatha02@example.net', datetime.datetime(2024, 6, 27, 21, 26, 10), None)
(4, 'Yasmin Cassiano', '98426072282', datetime.date(1942, 4, 21), 'Estação de Rezende, 866\nFlavio Marques Lisboa\n91310-865 Carvalho / RR', '04142637593', 'eduardo87@example.org', datetime.datetime(2024, 6, 27, 21, 26, 10), None)
(5, 'Sophia Moraes', '87064840280', datetime.date(1958, 8, 22), 'Largo de Cavalcanti, 84\nCidade

### Cartões

In [101]:
# cursor.execute('SELECT LAST_INSERT_ID()')
ultimo_id = len(clientes)
primeiro_id = 1

ids_clientes = list(range(primeiro_id, ultimo_id))

In [104]:
def gerar_numero_cartao():
    return ''.join([str(random.randint(0, 9)) for _ in range(16)])

In [106]:
sql = '''
INSERT INTO cartoes (ClienteID, NumeroCartao, CVV, DataValidade, Limite, DataEmissao)
VALUES (%s, %s, %s, %s, %s, %s)
'''

valores_cartoes = []

for cliente_id in ids_clientes:
    numero_cartao = gerar_numero_cartao()
    cvv = ''.join([str(random.randint(0, 9)) for _ in range(3)])
    data_validade = datetime.now() + timedelta(days=random.randint(365, 1825))
    limite = round(random.uniform(1000, 5000), 2)
    data_emissao = datetime.now()  
    
    valores_cartoes.append((cliente_id, numero_cartao, cvv, data_validade, limite, data_emissao))

cursor.executemany(sql, valores_cartoes)
conn.commit()
print("[] - Inserção de cartões concluída.")

Inserção de cartões concluída.


In [77]:
conn.close()

### Transações

In [107]:
# cursor.execute('SELECT LAST_INSERT_ID()')
ultimo_id = len(clientes)
primeiro_id = 1
min_t = 5
max_t = 10

ids_cartoes = list(range(primeiro_id, ultimo_id))

In [108]:
sql = '''
INSERT INTO transacoes (CartaoID, DataTransacao, Valor, Estabelecimento, Descricao)
VALUES (%s, %s, %s, %s, %s)
'''

valores_transacoes = []

for cartao_id in ids_cartoes:
    num_transacoes = random.randint(min_t, max_t)

    for _ in range(num_transacoes):
        data_transacao = datetime.now() - timedelta(days=random.randint(1, 365))
        valor = round(random.uniform(10.0, 500.0), 2)
        estabelecimento = f"Estabelecimento {random.randint(1, 20)}"
        descricao = f"Compra de {random.choice(['roupas', 'eletrônicos', 'alimentos', 'serviços'])}"

        valores_transacoes.append((cartao_id, data_transacao, valor, estabelecimento, descricao))

cursor.executemany(sql, valores_transacoes)
conn.commit()
print("[] - Inserção de transações concluída.")

[] - Inserção de transações concluída.
